## Initialize Environment
### This cell imports necessary libraries, initializes the BigQuery client,and sets up global variables for the analysis.

**IMPORTANT: In the cell below, you must enter your Google Cloud Project ID for the current lab. This value is crucial for accessing all resources within your lab environment.**

In [ ]:
# User: Please enter your Project ID in this cell.
PROJECT_ID = 'your-gcp-project-id' # <-- ENTER YOUR ACTUAL PROJECT ID HERE!

# Verify that PROJECT_ID is not empty. If it is, raise an error.
if not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please enter your Project ID above.")

print(f"Project ID set to: {PROJECT_ID}")

Now, run this cell to initialize the environment. This will import all necessary libraries, establish the connection to BigQuery, and define the key variables (such as GCS bucket paths) that will be used throughout the lab.

In [ ]:
# This cell imports necessary libraries, initializes the BigQuery client,
# and sets up global variables for the analysis.
from google.cloud import bigquery
import pandas as pd
from IPython.display import HTML, display, Image, Video
from google.cloud import storage
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure PROJECT_ID has been defined in the cell above.
if 'PROJECT_ID' not in locals() or not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please run the 'Set Your Project ID' cell above first.")

client = bigquery.Client(project=PROJECT_ID, location="us-central1") # Added project argument

# IMPORTANT: Verify this PROJECT_ID matches your lab's project ID.
DATASET_ID = 'cymbal'
REGION = 'us-central1'
CONNECTION_ID_FOR_EXTERNAL_TABLE = f'{REGION}.gemini_conn'
GEMINI_MODEL_NAME = f'{PROJECT_ID}.{DATASET_ID}.gemini_flash_model'
GCS_BUCKET_URI = f'gs://{PROJECT_ID}-bucket'
CSV_GCS_URI = f'{GCS_BUCKET_URI}/review/customer_reviews.csv'
IMAGES_GCS_URI_PATTERN = f'{GCS_BUCKET_URI}/review/images/*'
VIDEOS_GCS_URI_PATTERN = f'{GCS_BUCKET_URI}/review/videos/*'

# Create the dataset if it doesn't exist to avoid errors.
client.create_dataset(DATASET_ID, exists_ok=True)
print(f"Dataset {DATASET_ID} ensured.")
print(f"BigQuery Client Initialized. Project ID: {PROJECT_ID}")

def run_bq_query(sql: str, client: bigquery.Client):
    """A helper function to run BigQuery queries and return results."""
    try:
        query_job = client.query(sql)
        print(f"Job {query_job.job_id} in state {query_job.state}")
        if query_job.statement_type == 'SELECT':
            df = query_job.to_dataframe()
            print(f"Query complete. Fetched {len(df)} rows.")
            return df
        else:
            query_job.result()
            print(f"Query for statement type {query_job.statement_type} complete.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Create Text Reviews External Table
### We create an **EXTERNAL TABLE** that points directly to the CSV file in GCS with an explicitly defined schema. This method completely bypasses any data loading and schema auto-detection issues.

In [ ]:

table_id_reviews_external = f"{PROJECT_ID}.{DATASET_ID}.customer_reviews_external"
sql_create_external_table = f"""
CREATE OR REPLACE EXTERNAL TABLE `{table_id_reviews_external}` (
    customer_review_id INT64,
    customer_id INT64,
    location_id INT64,
    review_datetime DATETIME,
    review_text STRING,
    social_media_source STRING,
    social_media_handle STRING,
    product_id INT64,
    rating INT64
)
OPTIONS (
  format = 'CSV',
  uris = ['{CSV_GCS_URI}'],
  field_delimiter = ',',
  skip_leading_rows = 1,
  allow_quoted_newlines = TRUE
);
"""
print(f"Creating external table: {table_id_reviews_external}...")
run_bq_query(sql_create_external_table, client)


## Verify the Text Review Table

In [ ]:
%%bigquery
SELECT * FROM `cymbal.customer_reviews_external`
LIMIT 5

## Create Object Tables for Images and Videos

In [ ]:
# Creates an object table for review images.
table_id_review_images = f"{PROJECT_ID}.{DATASET_ID}.review_images"
sql_create_image_table = f"""
CREATE OR REPLACE EXTERNAL TABLE `{table_id_review_images}`
WITH CONNECTION `{CONNECTION_ID_FOR_EXTERNAL_TABLE}`
OPTIONS (object_metadata = 'SIMPLE', uris = ['{IMAGES_GCS_URI_PATTERN}']);
"""
print(f"\nCreating object table for review images: {table_id_review_images}")
run_bq_query(sql_create_image_table, client)

# Creates an object table for review videos.
table_id_review_videos = f"{PROJECT_ID}.{DATASET_ID}.review_videos"
sql_create_video_table = f"""
CREATE OR REPLACE EXTERNAL TABLE `{table_id_review_videos}`
WITH CONNECTION `{CONNECTION_ID_FOR_EXTERNAL_TABLE}`
OPTIONS (object_metadata = 'SIMPLE', uris = ['{VIDEOS_GCS_URI_PATTERN}']);
"""
print(f"\nCreating object table for review videos: {table_id_review_videos}")
run_bq_query(sql_create_video_table, client)

## Verify BigQuery Object Table (Review Images)

In [ ]:
%%bigquery
SELECT * FROM `cymbal.review_images`
LIMIT 5

## Verify BigQuery Object Table (Review Videos)

In [ ]:
%%bigquery
SELECT * FROM `cymbal.review_videos`
LIMIT 3

## Create a Gemini Model in BigQuery
### This SQL command creates a remote model in BigQuery, linking it to the Gemini Flash endpoint via the connection we set up earlier.

In [ ]:
sql_create_gemini_model = f"""
CREATE OR REPLACE MODEL `{GEMINI_MODEL_NAME}`
REMOTE WITH CONNECTION `{CONNECTION_ID_FOR_EXTERNAL_TABLE}`
OPTIONS (endpoint = 'gemini-2.0-flash-001');
"""
print(f"Creating Gemini model: {GEMINI_MODEL_NAME}...")
run_bq_query(sql_create_gemini_model, client)

## Analyze Text for Keywords and Sentiment
### Now that the source table is guaranteed to have the correct schema, we can use this simple and efficient 'pass-through' pattern. The model will process each review and pass through the 'customer_review_id' for easy joining later.

In [ ]:
# Analyze text for keywords
table_id_reviews_keywords = f"{PROJECT_ID}.{DATASET_ID}.customer_reviews_keywords"
sql_analyze_keywords = f"""
CREATE OR REPLACE TABLE `{table_id_reviews_keywords}` AS
SELECT
  customer_review_id,
  ml_generate_text_llm_result AS keywords_json_string
FROM ML.GENERATE_TEXT(
    MODEL `{GEMINI_MODEL_NAME}`,
    (
      SELECT
        customer_review_id,
        CONCAT('Extract keywords from the following customer review. Return as a JSON string array like {{"keywords": ["keyword1"]}}. Review: ', review_text) AS prompt
      FROM
        `{table_id_reviews_external}`
    ),
    STRUCT(0.2 AS temperature, TRUE AS flatten_json_output)
  );
"""
print("Starting customer review keyword analysis...")
run_bq_query(sql_analyze_keywords, client)


# Analyze text for sentiment
table_id_reviews_analysis = f"{PROJECT_ID}.{DATASET_ID}.customer_reviews_analysis"
sql_analyze_sentiment = f"""
CREATE OR REPLACE TABLE `{table_id_reviews_analysis}` AS
SELECT
  customer_review_id,
  ml_generate_text_llm_result AS sentiment_json_string
FROM ML.GENERATE_TEXT(
    MODEL `{GEMINI_MODEL_NAME}`,
    (
      SELECT
        customer_review_id,
        CONCAT('Classify the sentiment of the following review as "positive", "negative", or "neutral". Return as a JSON string like {{"sentiment": "positive"}}. Review: ', review_text) AS prompt
      FROM
        `{table_id_reviews_external}`
    ),
    STRUCT(0.2 AS temperature, TRUE AS flatten_json_output)
  );
"""
print("\nStarting customer review sentiment analysis...")
run_bq_query(sql_analyze_sentiment, client)

## Verify Text Analysis Results

In [ ]:
%%bigquery
SELECT * FROM `cymbal.customer_reviews_keywords`
LIMIT 5

In [ ]:
%%bigquery
SELECT * FROM `cymbal.customer_reviews_analysis`
LIMIT 5

## Analyze Images and Videos
### Analyze Images and Videos using Gemini, BigQuery SQL and Object Tables

In [ ]:
# Invokes Gemini to analyze the content of each image in the object table.
table_id_image_results = f"{PROJECT_ID}.{DATASET_ID}.review_images_results"
sql_analyze_images = f"""
CREATE OR REPLACE TABLE `{table_id_image_results}` AS
SELECT uri, ml_generate_text_llm_result AS image_analysis_json
FROM ML.GENERATE_TEXT( MODEL `{GEMINI_MODEL_NAME}`, TABLE `{table_id_review_images}`,
    STRUCT('For each image, summarize it and extract relevant keywords. Answer in JSON with keys "summary" and "keywords".' AS prompt, TRUE AS flatten_json_output)
);
"""
print("\nStarting image analysis...")
run_bq_query(sql_analyze_images, client)

# Invokes Gemini to analyze the content of each video in the object table.
table_id_video_results = f"{PROJECT_ID}.{DATASET_ID}.review_videos_results"
sql_analyze_videos = f"""
CREATE OR REPLACE TABLE `{table_id_video_results}` AS
SELECT uri, ml_generate_text_llm_result AS video_analysis_json
FROM ML.GENERATE_TEXT( MODEL `{GEMINI_MODEL_NAME}`, TABLE `{table_id_review_videos}`,
    STRUCT('For each video, summarize it and extract keywords. Answer in JSON with keys "summary" and "keywords".' AS prompt, TRUE AS flatten_json_output)
);
"""
print("\nStarting video analysis...")
run_bq_query(sql_analyze_videos, client)

## Review Image and Video Analysis Samples

In [ ]:
# This cell fetches and displays media files for direct comparison with the analysis results.
storage_client = storage.Client()

print(f"\n--- Displaying Individual Image Samples & Analysis ---")
df_img_samples = run_bq_query(f"SELECT uri, image_analysis_json FROM `{table_id_image_results}` LIMIT 2", client)
if df_img_samples is not None:
    for _, row in df_img_samples.iterrows():
        print("-" * 30)
        print(f"Analysis for: {row['uri']}")
        display(HTML(f"<pre style='white-space: pre-wrap;'>{row['image_analysis_json']}</pre>"))
        try:
            bucket_name, blob_name = row['uri'].replace("gs://", "").split("/", 1)
            display(Image(data=storage_client.bucket(bucket_name).blob(blob_name).download_as_bytes(), width=300))
        except Exception as e:
            print(f"--> Could not display image {row['uri']}. Error: {e}")

print(f"\n--- Displaying Individual Video Samples & Analysis ---")
df_vid_samples = run_bq_query(f"SELECT uri, video_analysis_json FROM `{table_id_video_results}` LIMIT 1", client)
if df_vid_samples is not None:
    for _, row in df_vid_samples.iterrows():
        print("-" * 30)
        print(f"Analysis for: {row['uri']}")
        display(HTML(f"<pre style='white-space: pre-wrap;'>{row['video_analysis_json']}</pre>"))

video_url=f"https://storage.googleapis.com/{PROJECT_ID}-bucket/review/videos/Review%20Video%20(1).mp4"
Video(video_url, width=640)

## Create a Unified Analysis Table
### Putting all together into BigQuery Multimodal Table

In [ ]:
# The regular expression in REGEXP_EXTRACT is corrected to have only one capturing group `(\\d+)`.
# This allows us to join the image/video analysis back to the original review by extracting the review ID from the filename.
table_id_multimodal_reviews = f"{PROJECT_ID}.{DATASET_ID}.multimodal_customer_reviews"
sql_create_multimodal_table = f"""
CREATE OR REPLACE TABLE `{table_id_multimodal_reviews}` AS
WITH
  image_results_parsed AS (
    SELECT SAFE_CAST(REGEXP_EXTRACT(uri, r'Review.*\\((\\d+)\\)') AS INT64) AS customer_review_id, uri AS image_uri, image_analysis_json
    FROM `{table_id_image_results}`
  ),
  video_results_parsed AS (
    SELECT SAFE_CAST(REGEXP_EXTRACT(uri, r'Video.*\\((\\d+)\\)') AS INT64) AS customer_review_id, uri AS video_uri, video_analysis_json
    FROM `{table_id_video_results}`
  )
SELECT
    cr.*, -- Select all columns from the correctly-defined source table
    s.sentiment_json_string,
    k.keywords_json_string,
    irp.image_uri,
    irp.image_analysis_json,
    vrp.video_uri,
    vrp.video_analysis_json
FROM `{table_id_reviews_external}` AS cr
LEFT JOIN `{table_id_reviews_analysis}` AS s ON cr.customer_review_id = s.customer_review_id
LEFT JOIN `{table_id_reviews_keywords}` AS k ON cr.customer_review_id = k.customer_review_id
LEFT JOIN image_results_parsed AS irp ON cr.customer_review_id = irp.customer_review_id
LEFT JOIN video_results_parsed AS vrp ON cr.customer_review_id = vrp.customer_review_id;
"""
print("Creating unified multimodal analysis table...")
run_bq_query(sql_create_multimodal_table, client)

## Verify the Unified Table

In [ ]:
%%bigquery
SELECT * FROM `cymbal.multimodal_customer_reviews` where video_uri is not null

## Visualize Sentiment Distribution with GenAI


- For this step, you will use the notebook's built-in generative AI assistant to create a plot for you.
- Click the **+ Code** button to add a new code cell.
- Inside the new cell, click the **Generate** button.
- In the prompt box, type the following as a comment:
   - `plot a bar chart for the distribution of text_sentiment in the multimodal_customer_reviews table`
- Accept the suggested code, then run the cell to display the chart. This provides a quick overview of the overall sentiment balance.


## Hands-On Lab: Generating Plots with GenAI


- Now it's your turn to use the notebook's built-in generative AI assistant. You will write a simple prompt to create a visualization yourself.
- Your challenge is to ask the generative AI assistant new and creative questions to uncover hidden patterns and insights from the `table_id_multimodal_reviews`.
- Below are some examples to inspire you. Try running these, and then create your own!

   1. Generate a line graph tracking the daily counts of positive, negative, and neutral reviews over time
   ```
   I want to analyze how customer sentiment has changed day by day.
   Select data from the table_id_multimodal_reviews table and generate a line chart that tracks the daily counts of positive, negative, and neutral sentiments.
   The sentiment is in the 'sentiment_json_string' field, and the date is in the 'review_datetime' field.
   ```
   2. Create a bar chart comparing the total count of reviews containing an image against the total count of reviews containing a video.
   ```
   Using table_id_multimodal_reviews, count the number of reviews that have an image and the number of reviews that have a video. Show the result as a bar chart.
   ```
   3. Plot a grouped bar chart showing the counts of positive, negative, and neutral reviews for customer age groups '18-29', '30-45', '46-60', and '61+'.
   ```
   I need a breakdown of sentiment by customer age group.
   First, join the `table_id_multimodal_reviews` table with the `customers` table using `customer_id`.
   Then, create four age groups from the `age` column: '18-29', '30-45', '46-60', and '61+'.
   Finally, create a grouped bar chart where each age group shows the total count of 'positive', 'negative', and 'neutral' sentiments.
   ```
   4. Generate a grouped bar chart comparing the total counts of positive, negative, and neutral reviews across all gender categories.
   ```
   I want to analyze if customer sentiment differs by gender.
   Join the `table_id_multimodal_reviews` table with the `customers` table using `customer_id`.
   For each gender, count the total number of 'positive', 'negative', and 'neutral' reviews.
   Present this comparison as a grouped bar chart, where each gender has its own set of bars for the sentiments.
   ```

